<a href="https://www.kaggle.com/code/logxdx158/flight-delay-prediction?scriptVersionId=209175433" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mini-flight-delay-prediction/flight_delays_test.csv
/kaggle/input/mini-flight-delay-prediction/flight_delays_train.csv


In [2]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

In [3]:

#  load dataset

df_train = pd.read_csv("/kaggle/input/mini-flight-delay-prediction/flight_delays_train.csv")

df_test = pd.read_csv("/kaggle/input/mini-flight-delay-prediction/flight_delays_test.csv")


In [4]:
df_train.info()

print()

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Month          100000 non-null  object
 1   DayofMonth     100000 non-null  object
 2   DayOfWeek      100000 

# EDA

In [5]:
print("train:")

print(df_train.head())

print("\n\ntest:")

print(df_test.head())

train:
  Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance  \
0   c-8       c-21       c-7     1934            AA    ATL  DFW       732   
1   c-4       c-20       c-3     1548            US    PIT  MCO       834   
2   c-9        c-2       c-5     1422            XE    RDU  CLE       416   
3  c-11       c-25       c-6     1015            OO    DEN  MEM       872   
4  c-10        c-7       c-6     1828            WN    MDW  OMA       423   

  dep_delayed_15min  
0                 N  
1                 N  
2                 N  
3                 N  
4                 Y  


test:
  Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance
0   c-7       c-25       c-3      615            YV    MRY  PHX       598
1   c-4       c-17       c-2      739            WN    LAS  HOU      1235
2  c-12        c-2       c-7      651            MQ    GSP  ORD       577
3   c-3       c-25       c-7     1614            WN    BWI  MHT       377
4   c-6        c-6  

In [6]:
print("Unique values of columns\n----------------------------")

for i in df_train.columns:

    print(i,":", len(df_train[i].unique()))

Unique values of columns
----------------------------
Month : 12
DayofMonth : 31
DayOfWeek : 7
DepTime : 1300
UniqueCarrier : 22
Origin : 289
Dest : 289
Distance : 1310
dep_delayed_15min : 2


In [7]:
encoding_labels = ['UniqueCarrier', 'Origin', 'Dest']

for i in encoding_labels:
    values, counts = np.unique(df_train[i], return_counts=True)
    print(f"\n\n{i}:\n---------------------")
    print("Data present in Train but not in test:")
    for j, c in zip(values, counts):
        if j not in  df_test[i].unique():
            print(f"{j}: {c}")

    values, counts = np.unique(df_test[i], return_counts=True)
    print("\nData present in test but not in train:")
    for j, c in zip(values, counts):
        if j not in  df_train[i].unique():
            print(f"{j}: {c}")



UniqueCarrier:
---------------------
Data present in Train but not in test:
DH: 966
HP: 1378
TZ: 446

Data present in test but not in train:
9E: 3461


Origin:
---------------------
Data present in Train but not in test:
CDC: 10
HKY: 7
HVN: 7
ISO: 8
PIE: 3
VCT: 1
VIS: 1

Data present in test but not in train:
ALO: 2
BJI: 3
CMX: 2
EAU: 1
ELM: 13
EWN: 7
HHH: 13
INL: 3
MTH: 3
PLN: 5
PMD: 4
RHI: 3
ROW: 3
SLE: 12
SOP: 2
SUX: 4
TTN: 8
YKM: 11


Dest:
---------------------
Data present in Train but not in test:
CDC: 7
GST: 2
HKY: 5
HVN: 10
ISO: 9
PIE: 5
TEX: 1
VCT: 1
VIS: 7

Data present in test but not in train:
ADK: 1
ALO: 3
BJI: 1
CMX: 2
EAU: 1
ELM: 14
EWN: 7
HHH: 23
INL: 3
MTH: 1
OAJ: 14
PLN: 5
PMD: 3
RHI: 4
ROW: 2
SLE: 3
SUX: 5
YKM: 6


In [8]:
for i in encoding_labels:
    print(i)
    # For values in train but not in test
    df_train[i] = df_train[i].where(df_train[i].isin(df_test[i].unique()), 'Unknown')
    
    # For values in test but not in train
    df_test[i] = df_test[i].where(df_test[i].isin(df_train[i].unique()), 'Unknown')


UniqueCarrier
Origin
Dest


In [9]:
# Function to remove 'c-' prefix from relevant columns
def clean_column_values(df, columns):
    for col in columns:
        df[col] = df[col].str.replace('c-', '', regex=False).astype(int)
    return df

# Clean the 'Month', 'DayofMonth', 'DayOfWeek' columns in the train and test datasets
columns_to_clean = ['Month', 'DayofMonth', 'DayOfWeek']
df_train = clean_column_values(df_train, columns_to_clean)
df_test = clean_column_values(df_test, columns_to_clean)

# Convert the target column ('dep_delayed_15min') to binary values (1 for 'Y', 0 for 'N')
df_train['dep_delayed_15min'] = df_train['dep_delayed_15min'].apply(lambda x: 1 if x == 'Y' else 0)

# Additional feature engineering (extract hour and minute from DepTime)
df_train['DepHour'] = df_train['DepTime'] // 100
df_train['DepMinute'] = df_train['DepTime'] % 100
df_test['DepHour'] = df_test['DepTime'] // 100
df_test['DepMinute'] = df_test['DepTime'] % 100


from sklearn.preprocessing import OneHotEncoder
# One-hot encode categorical variables
categorical_columns = ['Month', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_features_train = encoder.fit_transform(df_train[categorical_columns])
encoded_features_test = encoder.transform(df_test[categorical_columns])

# Create DataFrames for encoded features and join with original datasets
encoded_df_train = pd.DataFrame(encoded_features_train, columns=encoder.get_feature_names_out(categorical_columns))
encoded_df_test = pd.DataFrame(encoded_features_test, columns=encoder.get_feature_names_out(categorical_columns))

df_train = df_train.join(encoded_df_train)
df_test = df_test.join(encoded_df_test)

# Drop original categorical columns
df_train.drop(columns=categorical_columns + ['DepTime'], inplace=True)
df_test.drop(columns=categorical_columns + ['DepTime'], inplace=True)

# Optional: Distance binning
df_train['DistanceCategory'] = pd.cut(df_train['Distance'], bins=[0, 500, 1000, 1500, float('inf')], labels=[0, 1, 2, 3])
df_test['DistanceCategory'] = pd.cut(df_test['Distance'], bins=[0, 500, 1000, 1500, float('inf')], labels=[0, 1, 2, 3])

# Split dataset for training
X_train = df_train.drop(columns=['dep_delayed_15min'])
y_train = df_train['dep_delayed_15min']


y_train.value_counts()

dep_delayed_15min
0    80956
1    19044
Name: count, dtype: int64

In [10]:
df_train.head()

,DayofMonth,Distance,dep_delayed_15min,DepHour,DepMinute,Month_2,Month_3,Month_4,Month_5,Month_6,...,Dest_TYS,Dest_Unknown,Dest_VLD,Dest_VPS,Dest_WRG,Dest_WYS,Dest_XNA,Dest_YAK,Dest_YUM,DistanceCategory
0,21,732,0,19,34,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,20,834,0,15,48,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,2,416,0,14,22,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,25,872,0,10,15,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,7,423,1,18,28,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [11]:
df_test.head()

,DayofMonth,Distance,DepHour,DepMinute,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,...,Dest_TYS,Dest_Unknown,Dest_VLD,Dest_VPS,Dest_WRG,Dest_WYS,Dest_XNA,Dest_YAK,Dest_YUM,DistanceCategory
0,25,598,6,15,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,17,1235,7,39,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,2,577,6,51,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,25,377,16,14,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,6,258,15,5,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [12]:
np.unique(df_train.isnull().any()), np.unique(df_test.isnull().any())

(array([False]), array([False]))

In [13]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(df_test)

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

np.unique(np.isnan(X_train)), np.unique(np.isnan(X_resampled))

(array([False]), array([False]))

In [15]:
import os
import shutil

working_dir = "/kaggle/working/"

# Remove all files in the working directory
for filename in os.listdir(working_dir):
    file_path = os.path.join(working_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove file or link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directory and its contents
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")

print("Kaggle working directory cleared.")


Kaggle working directory cleared.


# Training & Validation

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=42)
X_resampled, X_val_resampled, y_resampled, y_val_resampled = train_test_split(X_resampled, y_resampled, stratify=y_resampled, test_size=0.2, random_state=42)

In [17]:
import pickle
from sklearn.metrics import accuracy_score, f1_score
import os

# Load each model and evaluate on the test data
def load_and_evaluate(model_file, X_test, y_test, fname, dataset):
    # Load the model from the pickle file
    with open(model_file, "rb") as file:
        model = pickle.load(file)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    with open(f"{working_dir}{fname}.txt", "a") as file:
        file.write(f"Model: {model_file}\n")
        file.write(f"Dataset: {dataset}\n")
        file.write(f"Accuracy: {accuracy:.4f}\n")
        file.write(f"F1 Score: {f1:.4f}\n")
        file.write("-" * 30 + "\n")
    
    # Print the results
    print(f"Model: {model_file}")
    print(f"Dataset: {dataset}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("-" * 30)

In [18]:
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import multiprocessing as mp
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

models = {
    "DecisionTree": DecisionTreeClassifier(max_depth=15, min_samples_split=5),
    "RandomForest": RandomForestClassifier(max_depth=23, n_estimators=100),
    "AdaBoost": AdaBoostClassifier(n_estimators=500),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, max_depth=3),
    "GaussianNB": GaussianNB(),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(n_neighbors=5),    
    "ExtraTrees": ExtraTreesClassifier(n_estimators=100, max_depth=10),
}

# Function to train and save each model
def train_and_save(model_name, model, X, y, suffix):
    model.fit(X, y)
    with open(f"{working_dir}{model_name}_{suffix}.pkl", "wb") as file:
        pickle.dump(model, file)
    print(f"{model_name} trained on {suffix} data and saved.")

# Prepare data for multiprocessing
tasks = []
for model_name, model in models.items():
    # Add tasks for both original and resampled data
    tasks.append((model_name, model, X_train, y_train, "Original"))
    tasks.append((model_name, model, X_resampled, y_resampled, "Resampled"))

# Use multiprocessing to run tasks in parallel
if __name__ == "__main__":
    with mp.Pool(processes=min(mp.cpu_count(), 8)) as pool:
        pool.starmap(train_and_save, tasks)

    print("All models have been trained and saved on both original and resampled data.")


DecisionTree trained on Original data and saved.
DecisionTree trained on Resampled data and saved.
GaussianNB trained on Original data and saved.
GaussianNB trained on Resampled data and saved.
RandomForest trained on Original data and saved.
LDA trained on Original data and saved.
LDA trained on Resampled data and saved.
RandomForest trained on Resampled data and saved.


/opt/conda/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QDA trained on Original data and saved.


/opt/conda/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


LogisticRegression trained on Original data and saved.
QDA trained on Resampled data and saved.
KNN trained on Original data and saved.
KNN trained on Resampled data and saved.
LogisticRegression trained on Resampled data and saved.
ExtraTrees trained on Original data and saved.
ExtraTrees trained on Resampled data and saved.
GradientBoosting trained on Original data and saved.
AdaBoost trained on Original data and saved.
GradientBoosting trained on Resampled data and saved.
AdaBoost trained on Resampled data and saved.
All models have been trained and saved on both original and resampled data.


In [19]:
model_file_paths = [f"{working_dir}{model_name}_{suffix}.pkl" for model_name in models.keys() for suffix in ["Original", "Resampled"]]

# Evaluate on the test data
if __name__ == "__main__":
    # Assuming X_test and y_test are already defined (from your test data)
    # Load and evaluate each model
    c=0
    for model_file in model_file_paths:
        c+=1
        
        load_and_evaluate(model_file, X_train, y_train, "train", "Original")
        load_and_evaluate(model_file, X_resampled, y_resampled, "train", "Resampled")
        if c%2==0:
            print()
            with open(f"{working_dir}train.txt", "a") as file:
                file.write("\n")

        load_and_evaluate(model_file, X_val, y_val, "validation", "Original")
        load_and_evaluate(model_file, X_val_resampled, y_val_resampled, "validation", "Resampled")
        if c%2==0:
            print()
            with open(f"{working_dir}validation.txt", "a") as file:
                file.write("\n")

    print("Evaluation of all models completed.")


Model: /kaggle/working/DecisionTree_Original.pkl
Dataset: Original
Accuracy: 0.8510
F1 Score: 0.4132
------------------------------
Model: /kaggle/working/DecisionTree_Original.pkl
Dataset: Resampled
Accuracy: 0.6000
F1 Score: 0.3482
------------------------------
Model: /kaggle/working/DecisionTree_Original.pkl
Dataset: Original
Accuracy: 0.8129
F1 Score: 0.2628
------------------------------
Model: /kaggle/working/DecisionTree_Original.pkl
Dataset: Resampled
Accuracy: 0.5871
F1 Score: 0.3384
------------------------------
Model: /kaggle/working/DecisionTree_Resampled.pkl
Dataset: Original
Accuracy: 0.8210
F1 Score: 0.2247
------------------------------
Model: /kaggle/working/DecisionTree_Resampled.pkl
Dataset: Resampled
Accuracy: 0.8631
F1 Score: 0.8446
------------------------------

Model: /kaggle/working/DecisionTree_Resampled.pkl
Dataset: Original
Accuracy: 0.8124
F1 Score: 0.2101
------------------------------
Model: /kaggle/working/DecisionTree_Resampled.pkl
Dataset: Resampled


In [20]:
# predictions on test set

for model_file in model_file_paths:
    with open(model_file, "rb") as file:
        model = pickle.load(file)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)

    with open(f"{working_dir}predictions.txt", 'a') as file:
        file.write(f"{y_pred}\n")


In [21]:
!zip -v -r models.zip "/kaggle/working/"

  adding: kaggle/working/	(in=0) (out=0) (stored 0%)
  adding: kaggle/working/LDA_Original.pkl	(in=24985) (out=16674) (deflated 33%)
  adding: kaggle/working/DecisionTree_Original.pkl 	(in=228704) (out=41623) (deflated 82%)
  adding: kaggle/working/KNN_Original.pkl ....................................	(in=386560727) (out=2616758) (deflated 99%)
  adding: kaggle/working/GaussianNB_Original.pkl	(in=19882) (out=10884) (deflated 45%)
  adding: kaggle/working/QDA_Resampled.pkl 	(in=5837763) (out=5527935) (deflated 5%)
  adding: kaggle/working/predictions.txt	(in=360) (out=46) (deflated 87%)
  adding: kaggle/working/GradientBoosting_Resampled.pkl 	(in=124111) (out=40939) (deflated 67%)
  adding: kaggle/working/train.txt	(in=5230) (out=624) (deflated 88%)
  adding: kaggle/working/ExtraTrees_Resampled.pkl 	(in=2281369) (out=613111) (deflated 73%)
  adding: kaggle/working/ExtraTrees_Original.pkl 	(in=3040075) (out=738245) (deflated 76%)
  adding: kaggle/working/GradientBoosting_Original.pkl 	(i